In [ ]:
import streamlit as st
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import streamlit as st
import pickle
from sklearn.metrics.pairwise import cosine_similarity
ratings=pd.read_csv('ratings.csv')
movies=pd.read_csv('movies.csv')

users_items = pd.pivot_table(data=ratings, values='rating', index='userId', columns='movieId')
users_items.fillna(0, inplace=True)

user_similarities = pd.DataFrame(cosine_similarity(users_items),
                                 columns=users_items.index, 
                                 index=users_items.index)

top_10 = pickle.load(open('D:/WBS Data Science Bootcamp/Recommender Systems/Popular_movies.sav', 'rb'))

st.title("WBSFLIX")
with st.container():
    st.subheader("Our Customers Favorites...")
    movie_names=list(top_10['title'][0:5])
    st.write(movie_names)



def weighted_user_rec(user_id, n):
  weights =(user_similarities.query("userId!=@user_id")[user_id] / sum(user_similarities.query("userId!=@user_id")[user_id]) )
  not_visited_restaurants = users_items.loc[users_items.index!=user_id, users_items.loc[user_id,:]==0]
  weighted_averages = pd.DataFrame(not_visited_restaurants.T.dot(weights), columns=["predicted_rating"])
  recommendations = weighted_averages.merge(movies, left_index=True, right_on="movieId")
  result=recommendations.sort_values("predicted_rating", ascending=False).head(n)
  return result

        
user_id = st.number_input(label="Hi! I'm your personal recommender. Tell me your userID",min_value=1,step=1)
#submit_button = st.form_submit_button(label='Submit')
result = weighted_user_rec(user_id, 5)
st.write("You will probably like the movies:") 
final=list(result['title'])[0:5]
st.write(final)